In [1]:
%load_ext autoreload
%autoreload 2
from helpers import *
from networks import *
from tqdm import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt

/Users/wlan/miniforge3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define some tunable variables

In [2]:
batch_size = 5 # > 10 is bad
lr = 0.001
mom = 0.5
epochs = 50
networ_type = 'CNN1' #FNN CNN1 CNN2
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [3]:
device

device(type='mps')

### Prepare data for training

In [4]:
train_x, train_y, test_x, test_y = init_data("train.csv", 0.7)
train = MyDataset(train_x, train_y)
test = MyDataset(test_x, test_y)
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

In [5]:
num_features = train.x.shape[1]
num_features

305

### Build the neural network

In [6]:
net = eval(networ_type)()
net.to(device)
net

CNN1(
  (bn0): BatchNorm1d(305, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop0): Dropout(p=0.25, inplace=False)
  (prep): Linear(in_features=305, out_features=1000, bias=True)
  (bn1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv1d(50, 100, kernel_size=(5,), stride=(1,), padding=(2,), groups=50)
  (pool1): AdaptiveAvgPool1d(output_size=10)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop2): Dropout(p=0.25, inplace=False)
  (conv2): Conv1d(100, 100, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop3): Dropout(p=0.25, inplace=False)
  (conv3): Conv1d(100, 100, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), groups=1

### Train

In [7]:
criterion = nn.MSELoss()
# criterion = nn.L1Loss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=mom)

In [8]:
los = []
tlos = []
for epoch in range(epochs):
    tot_loss = 0.0
    net.train()
    for i, (inputs, labels) in enumerate(tqdm(train_loader), 0):
        inputs, labels = inputs.to(device), labels.to(device) # Move to GPU / CPU
        optimizer.zero_grad() # zero the parameter gradients

        labels = labels.unsqueeze(-1) # Batch color value
        # forward + backward + optimize
        # inputs = inputs.unsqueeze(1)
        outputs = net(inputs)
        # print(outputs.shape, labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        tot_loss += loss.item()
    # print(tot_loss)
    ttot_loss = 0.0
    net.eval()
    for i, (inputs, labels) in enumerate(test_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device) # Move to GPU / CPU
        with torch.no_grad():
            labels = labels.unsqueeze(-1) # Batch color value
            # inputs = inputs.unsqueeze(1) #.to_sparse()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
        ttot_loss += loss.item()
    los.append(tot_loss)
    tlos.append(ttot_loss)
    print(epoch, tot_loss, ttot_loss)

100%|██████████| 205/205 [00:02<00:00, 76.67it/s] 


0 149.00148969516158 1314131.1241624206


100%|██████████| 205/205 [00:01<00:00, 122.77it/s]


1 120.93343037553132 2157674.2691277955


100%|██████████| 205/205 [00:01<00:00, 122.87it/s]


...

47 86.40182356350124 82913.11425922066


100%|██████████| 205/205 [00:01<00:00, 122.85it/s]


48 86.03145030263113 477491.4327265993


100%|██████████| 205/205 [00:01<00:00, 123.72it/s]


49 77.0422130394727 1078744.2192449383


In [1]:
plt.plot(los, label="train")
plt.plot(tlos, label="test")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

NameError: name 'plt' is not defined

In [10]:
net.eval()
all_loss_train = []
with torch.no_grad():
    tot_loss = 0
    for i, (inputs, labels) in enumerate(tqdm(train_loader), 0):
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.unsqueeze(1) #.to_sparse()
        outputs = net(inputs).ravel()
        all_loss_train.extend((abs(outputs - labels) / abs(labels)).tolist())
        

  0%|          | 0/205 [00:00<?, ?it/s]


RuntimeError: running_mean should contain 1 elements not 305

In [ ]:
print(test.x.shape, test.x.device)
print(test.y.shape, test.y.device)

In [ ]:
all_loss_test = []
with torch.no_grad():
    tot_loss = 0
    for i, (inputs, labels) in enumerate(tqdm(test_loader), 0):
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.unsqueeze(1)
        outputs = net(inputs).ravel()
        all_loss_test.extend((abs(outputs - labels) / abs(labels)).tolist())
    

In [ ]:
all_loss_train = np.array(all_loss_train)
all_loss_test = np.array(all_loss_test)


In [ ]:
len(np.where(all_loss_train < 1)[0]) / len(all_loss_train), len(np.where(all_loss_test < 1)[0]) / len(all_loss_test)

In [ ]:
good_pred_train = all_loss_train[np.where(all_loss_train < 1)]
good_pred_test = all_loss_test[np.where(all_loss_test < 1)]
good_pred_train.mean(), good_pred_train.std(), good_pred_test.mean(), good_pred_test.std()

In [ ]:
bins = range(101)
plt.hist([good_pred_train*100, good_pred_test*100], bins=bins, alpha=0.5, label=["train", "test"])
plt.legend()
plt.show()